In [13]:
#Joydip_Talukder_Task 3 : Modular Signature Scheme Verification Algorithm (Open task - Any coding language and any signature scheme can be used)
def signature_verifier(signer: str, data: bytes, signed_hash: bytes, scheme_type: str) -> bool:
    pass

In [5]:
# Assuming the ECDSA signature format
# First, define 'data'. Replace with the actual signature data
data = b'\x00\x01\x02...'  # Example: Replace with your actual data

# Now you can access 'data'
v = data[-1]         # recovery id
r = int.from_bytes(data[:32], 'big')
s = int.from_bytes(data[32:64], 'big')

In [7]:
!pip install ecdsa
import ecdsa

def verify_ecdsa_signature(signer, signed_hash, r, s, v):
    # Initialize the ECDSA key and verify signature
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(signer), curve=ecdsa.SECP256k1)
    try:
        return vk.verify_digest((r, s), signed_hash)
    except ecdsa.BadSignatureError:
        return False


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 4.3 MB/s eta 0:00:00


In [8]:
import ecdsa

def signature_verifier(signer: str, data: bytes, signed_hash: bytes, scheme_type: str) -> bool:
    if scheme_type == "ecdsa":
        # Parse the signature for ECDSA (assuming a common structure with r, s, v)
        r = int.from_bytes(data[:32], 'big')
        s = int.from_bytes(data[32:64], 'big')
        v = data[-1]  # recovery id

        return verify_ecdsa_signature(signer, signed_hash, r, s, v)

    elif scheme_type == "schnorr":
        # Schnorr verification logic would go here if implemented
        pass

    else:
        # Unsupported signature scheme
        return False

def verify_ecdsa_signature(signer, signed_hash, r, s, v):
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(signer), curve=ecdsa.SECP256k1)
    try:
        return vk.verify_digest((r, s), signed_hash)
    except ecdsa.BadSignatureError:
        return False


In [12]:
import ecdsa
import hashlib

# ... (other functions) ...

def verify_ecdsa_signature(signer, signed_hash, r, s, v):
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(signer), curve=ecdsa.SECP256k1)
    try:
        # Convert r and s to bytes and concatenate them
        signature = r.to_bytes(32, 'big') + s.to_bytes(32, 'big')
        return vk.verify_digest(signature, signed_hash, sigdecode=ecdsa.util.sigdecode_string) # Pass sigdecode argument
    except ecdsa.BadSignatureError:
        return False



# Generate a test ECDSA key pair
def generate_test_keypair():
    sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    vk = sk.verifying_key
    return sk, vk

# Create a test case for valid ECDSA signature
def test_valid_ecdsa_signature():
    sk, vk = generate_test_keypair()
    signer = vk.to_string().hex()  # Mocked public key as hex
    message = b"Test message"
    signed_hash = hashlib.sha256(message).digest()  # Hash of the message

    # Sign the message hash
    signature = sk.sign_digest(signed_hash)
    # Split the signature into r and s values for ECDSA format
    r = signature[:32]
    s = signature[32:64]
    v = b'\x00'  # Example recovery ID (not used in Python ecdsa lib)

    data = r + s + v  # Concatenate r, s, and v for test case

    # Verify using our function
    is_valid = signature_verifier(signer, data, signed_hash, "ecdsa")
    print("Test valid ECDSA signature:", "Passed" if is_valid else "Failed")

# Create a test case for invalid ECDSA signature
def test_invalid_ecdsa_signature():
    sk, vk = generate_test_keypair()
    signer = vk.to_string().hex()
    message = b"Test message"
    signed_hash = hashlib.sha256(message).digest()

    # Sign the message hash
    signature = sk.sign_digest(signed_hash)
    r = signature[:32]
    s = signature[32:64]
    v = b'\x01'  # Incorrect recovery ID for invalid test

    data = r + s + v  # Concatenate r, s, and v

    # Verify with an incorrect v to simulate an invalid signature
    is_valid = signature_verifier(signer, data, signed_hash, "ecdsa")
    print("Test invalid ECDSA signature:", "Passed" if not is_valid else "Failed")

# Create a test case for unsupported scheme
def test_unsupported_scheme():
    sk, vk = generate_test_keypair()
    signer = vk.to_string().hex()
    message = b"Test message"
    signed_hash = hashlib.sha256(message).digest()

    # Sign the message hash
    signature = sk.sign_digest(signed_hash)
    r = signature[:32]
    s = signature[32:64]
    v = b'\x00'

    data = r + s + v

    # Attempt to verify with unsupported scheme
    is_valid = signature_verifier(signer, data, signed_hash, "schnorr")
    print("Test unsupported scheme:", "Passed" if not is_valid else "Failed")

# Run the test cases
test_valid_ecdsa_signature()
test_invalid_ecdsa_signature()
test_unsupported_scheme()


Test valid ECDSA signature: Failed
Test invalid ECDSA signature: Passed
Test unsupported scheme: Passed
